In [ ]:
! pip uninstall -y tensorflow_datasets
! pip install tensorflow_datasets==4.4.0
!pip install -U keras-efficientnet-v2

In [ ]:
#This notebook highly depends on https://www.kaggle.com/code/tchaye59/efficientnet-tensorflow-baseline-tpu/notebook 
from kaggle_datasets import KaggleDatasets
GCS_PATH = KaggleDatasets().get_gcs_path()

In [ ]:
GCS_PATH

In [ ]:
import tensorflow as tf
import pandas as pd
import keras_efficientnet_v2
import os

# proj_dir = "/kaggle/input/sorghum-id-fgvc-9/"
#proj_dir = "C:/Users/Owner/Documents/dev/sorghum/"

df = pd.read_csv("/kaggle/input/sorghum-divided-512x512/512_separated_train_cultivar_mapping.csv")

In [ ]:
%%writefile fgvc_dataset.py -a
import tensorflow_datasets as tfds
import tensorflow as tf


class FGVCDataset(tfds.core.GeneratorBasedBuilder):
    VERSION = tfds.core.Version('0.1.0')
    
    def _split_generators(self, dl_manager):
        arr = [
            tfds.core.SplitGenerator(name=f'train',gen_kwargs={"split":"train"}),
            tfds.core.SplitGenerator(name=f'test',gen_kwargs={"split":"test"})
        ]
        return arr
    
    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            description=(""),
            #disable_shuffling=True,
            features=tfds.features.FeaturesDict({
                "img": tfds.features.Image(encoding_format='jpeg'),#dtype=tf.uint8,shape=(self.WIDTH,self.HEIGHT,3),
                "name": tfds.features.Tensor(dtype=tf.string,shape=()),
                "cultivar": tfds.features.Tensor(dtype=tf.string,shape=()),
                "target": tfds.features.Tensor(dtype=tf.int32,shape=()),
            }),
        )
    
    def _generate_examples(self,**args):
        print(args)
        split = args["split"]
        
        if split == 'train':
            for i in range(len(self.train_df)):
                row = self.train_df.iloc[i]
                img = row.fullpath
                yield i, {
                    'img':img,
                    'cultivar':row.cultivar,
                    'name':row.image,
                    'target':row.target,
                }
                
        if split == 'test':
            for i in range(len(self.test_df)):
                row = self.test_df.iloc[i]
                img = row.fullpath
                yield i, {
                    'img':img,
                    'name':row.image,
                    'cultivar':'',
                    'target':-1,
                }

In [ ]:
def auto_select_accelerator():
    TPU_DETECTED = False
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
        TPU_DETECTED = True
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy, TPU_DETECTED

strategy, IS_TPU_ENABLE = auto_select_accelerator()

In [ ]:
BATCH_SIZE = 128
AUTOTUNE = tf.data.experimental.AUTOTUNE
SHUFFLE_SIZE = 20000
WIDTH = 512
HEIGHT = 512
EPOCHS = 20
model_name = "sep_512_effnet.h5"

paths = df["fullpath"]
labels_str = df["cultivar"]
image_count = len(paths)
label_to_index = dict((name, index) for index,name in enumerate(labels_str.unique()))
labels_idx = labels_str.map(lambda x: label_to_index[x])

In [ ]:
from fgvc_dataset import FGVCDataset

data_dir= GCS_PATH
builder = FGVCDataset(data_dir=data_dir)
builder.download_and_prepare()
train_ds = builder.as_dataset()['train']
test_ds = builder.as_dataset()['test']

In [ ]:
def data_augmentation():
    return tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.05,width_factor=0.05),
#         tf.keras.layers.experimental.preprocessing.RandomCrop(height=HEIGHT, width=WIDTH),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
        tf.keras.layers.experimental.preprocessing.RandomFlip(),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.3),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.3,width_factor=0.3),
    ])
data_aug = data_augmentation()

def load_train_image(data):
    img = data['img']
    cultivar = data['cultivar']
    name = data['name']
    target = data['target']
    # Resize image
    img = tf.image.resize(img,(WIDTH, HEIGHT),)

    return img,target

def load_test_image(data):
    img = data['img']
    name = data['name']
    # Resize image
    img = tf.image.resize(img,(WIDTH, HEIGHT),)
#     img = tf.image.random_crop(value=img,size=(WIDTH, HEIGHT, 3))
#     img = resize_with_crop_or_pad(img, WIDTH, HEIGHT) 
#     img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img,name

def prepare_train_dataset(train_ds):
    steps = len(train_ds)//BATCH_SIZE
    train_ds = train_ds.repeat().shuffle(SHUFFLE_SIZE).map(load_train_image,num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.batch(BATCH_SIZE).map(lambda x,y:(data_aug(x),y),num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
    return steps,train_ds

def prepare_submission_dataset(ds):
    ds = ds.map(load_test_image,num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

In [ ]:
def create_model(steps):
    basemodel = keras_efficientnet_v2.EfficientNetV2XL(input_shape=(WIDTH, HEIGHT, 3), drop_connect_rate=0.2, num_classes=0, include_preprocessing=True, pretrained="imagenet21k-ft1k")
    basemodel.trainable = True
    
    image_input = tf.keras.layers.Input(shape=(WIDTH,HEIGHT,3))
    out = basemodel(image_input)
    out = tf.keras.layers.GlobalAveragePooling2D()(out)
    out = tf.keras.layers.Dropout(0.3)(out)
    out = tf.keras.layers.Dense(len(label_to_index), activation="softmax")(out)

    model = tf.keras.Model(image_input, out)

    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss='sparse_categorical_crossentropy',
                  metrics=["accuracy"],
                  steps_per_execution=steps)
    
    return model

In [ ]:
if IS_TPU_ENABLE:
    with strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
        model = create_model(32)
else:
    model = create_model(None)
        
model.summary()

In [ ]:
with strategy.scope():
    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy',mode='max', patience=20)
    ckp_callback = tf.keras.callbacks.ModelCheckpoint(
                                                filepath=f'model.h5',
                                                save_weights_only=True,
                                                monitor='accuracy',
                                                mode='max',
                                                options=tf.train.CheckpointOptions(experimental_io_device='/job:localhost'),
                                                save_best_only=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',mode='max',factor=0.2,patience=3, min_lr=1e-5)
    callbacks=[callback,ckp_callback,reduce_lr]
    
    steps_per_epoch,ds =  prepare_train_dataset(train_ds)

    history = model.fit(ds,
                        steps_per_epoch=steps_per_epoch,
                        epochs=10,
                        callbacks=callbacks)

In [ ]:
model.save("sep_512_effnet_30.h5")

In [ ]:
def predict(x):
    return model(x,training=False)
@tf.function
def dist_predict(dist_inputs):
    res = strategy.run(predict, args=(dist_inputs,))
    return res

In [ ]:
import numpy as np
from tqdm import tqdm 
with strategy.scope():
    ds = prepare_submission_dataset(test_ds)
    dist_ds = strategy.experimental_distribute_dataset(ds)
    
    all_names = []
    all_targets = []
    for img,names in tqdm(dist_ds):
        preds = dist_predict(img)
#         preds = tf.concat(preds.values,axis=0)
        preds = np.concatenate(preds.values ,axis=0) 
#         names = tf.concat(names.values,axis=0)
        names = np.concatenate(names.values ,axis=0) 
#         preds = preds.numpy()
#         names = names.numpy()
        all_targets.extend(list(preds))
        all_names.extend([s.decode('ascii') for s in names])

In [ ]:
df = pd.concat([pd.Series(all_names, name="sep_images"), pd.DataFrame(all_targets, columns=label_to_index.keys())], axis=1)
df

In [ ]:
df.to_csv("20220403_512_preds_30epoch.csv")

In [ ]:
df["filename_org"] = df['sep_images'].str.extract(r'([0-9]+_)', expand=False).str[:-1] + ".png"
submission = df.groupby('filename_org').sum()
submission.idxmax(axis=1).rename({"filename_org": "filename", "0":"cultivar"}).to_csv("submission_512_30epochs_20220403.csv")